### 데이터베이스 연결

In [1]:
%pip install pymysql        # !pip보다 %pip를 권장

                                              0.0/44.8 kB ? eta -:--:--
     ---------------------------------------- 44.8/44.8 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [32]:
import pymysql

In [33]:
with open('./db_config', 'r') as f:
    config = f.readlines()

In [34]:
config_dict = {}
for l in config:
    # k, v = l.split('=')         # =을 기준으로 key, value 분리
    idx = l.index('=')
    k = l[:idx]
    v = l[idx+1:]
    config_dict[k] = v.strip()  # 공백 문자 제거 후 config_dict의 key 별로 value 지정
config_dict["port"] = 3307
config_dict

{'host': 'localhost',
 'port': 3307,
 'user': 'leoni',
 'password': 'password',
 'database': 'study',
 'charset': 'utf8mb4'}

In [35]:
# config file 불러오는 함수

def read_config(p:str) -> dict:     # 코드 가독성을 위해 type 명시
    """
    p: config file 경로
    """
    with open(p, 'r') as f:
        lines = f.readlines()
    config_dict = {}

    for l in lines:
        idx = l.index('=')
        k = l[:idx]
        v = l[idx+1:]
        config_dict[k] = v.strip()
    config_dict["port"] = 3307
    
    return config_dict

In [45]:
db_config = read_config('./db_config')
db_config

{'host': 'localhost',
 'port': 3307,
 'user': 'leoni',
 'password': 'password',
 'database': 'study',
 'charset': 'utf8mb4'}

In [46]:
try:
    # conn = pymysql.connect(
    #     host="localhost",
    #     port=3307,
    #     user="leoni",
    #     password="password",
    #     database="study",
    #     charset="utf8mb4"
    # )
    conn = pymysql.connect(**db_config)
    # conn = pymysql.connect(**config_dict)
    print("연결 성공")
except Exception as e:
    print("연결 실패", e)

연결 성공


In [39]:
conn

### INSERT

In [53]:
# 방법 1 - execute 인자로 value 지정
with conn.cursor() as cur:
    sql = 'INSERT INTO `Students` (`Name`, `Age`, `Address`) VALUES (%s, %s, %s)'
    cur.execute(sql, ("손준현", "27", "서울"))       # 인자 작성 시 tuple로 작성하기 !

# 방법 2 - INSERT 쿼리문 내에서 value 지정
with conn.cursor() as cur:
    sql = 'INSERT INTO `Students` (`Name`, `Age`, `Address`) VALUES ("손준현", "27", "서울")'
    cur.execute(sql)

conn.commit()

### INSERT MANY

In [55]:
with conn.cursor() as cur:
    sql = 'INSERT INTO `Students` (`Name`) VALUES (%s)'
    cur.executemany(sql, [("손흥민", ), 
                             ("김민재", ), 
                             ("이강인", )])

conn.commit()

### SELECT

In [56]:
with conn.cursor() as cur:
    sql = 'SELECT * FROM Students'
    cur.execute(sql)
    data = cur.fetchall()

data

((1, '홍길동', 30, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 27, '경기'),
 (5, '손준현', 27, '서울'),
 (9, '손흥민', None, None),
 (10, '김민재', None, None),
 (11, '이강인', None, None))

In [68]:
page_size = 5
page = 1

limit = page_size
offset = page_size * (page - 1)

with conn.cursor() as cur:
    sql = 'SELECT * FROM Students LIMIT %s OFFSET %s'
    cur.execute(sql, (limit, offset))        # LIMIT 5 OFFSET 0
    data = cur.fetchall()

data

((1, '홍길동', 30, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 27, '경기'),
 (5, '손준현', 27, '서울'))

### 페이지 번호에 따라 조회되는 데이터 출력하는 함수

In [69]:
def pagination(page, page_size=5):
    limit = page_size
    offset = page_size * (page - 1)

    with conn.cursor() as cur:
        sql = 'SELECT * FROM Students LIMIT %s OFFSET %s'
        cur.execute(sql, (limit, offset))        # LIMIT 5 OFFSET 0
        data = cur.fetchall()
    return data

page_one = pagination(1)
page_one

((1, '홍길동', 30, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 27, '경기'),
 (5, '손준현', 27, '서울'))

### UPDATE

In [72]:
with conn.cursor() as cur:
    sql = 'UPDATE `Students` SET `Age`=%s WHERE `StudentID`=%s'
    cur.execute(sql, (26, 5))    # 순서 중요!
                   
conn.commit()

In [74]:
pagination(1)

((1, '홍길동', 30, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 28, '경기'),
 (5, '손준현', 26, '서울'))

In [73]:
with conn.cursor() as cur:
    sql = 'UPDATE `Students` SET `Age`=%s WHERE `StudentID`=%s'
    cur.executemany(sql, [
        (26, 5),
        (28, 4)
        ])
                   
conn.commit()

In [77]:
pagination(1, page_size=8)

((1, '홍길동', 30, '인천'),
 (2, '이연걸', 60, '서울'),
 (3, '이몽룡', 42, '대전'),
 (4, '성춘향', 28, '경기'),
 (5, '손준현', 26, '서울'),
 (6, '손흥민', None, None),
 (7, '김민재', None, None),
 (8, '이강인', None, None))

### DELETE

In [78]:
with conn.cursor() as cur:
    sql = 'DELETE FROM `Students` WHERE `StudentID`=%s'
    cur.execute(sql, (1, ))     # StudentID가 1인 행 삭제

conn.commit()

In [79]:
with conn.cursor() as cur:
    sql = 'DELETE FROM `Students` WHERE `StudentID` IN (%s, %s, %s)'
    cur.execute(sql, (2, 3, 4))     # StudentID가 2, 3, 4인 행 삭제

conn.commit()

# Foreign Key 제약 조건 해제
# SET FOREIGN_KEY_CHECKS = 0;

In [80]:
pagination(1)

((5, '손준현', 26, '서울'),
 (6, '손흥민', None, None),
 (7, '김민재', None, None),
 (8, '이강인', None, None))

In [ ]:
def delete(target_ids:list):
    with conn.cursor() as cur:
        sql = 'DELETE FROM `Students` WHERE `StudentID` IN (' + ','.join(['%s'] * len(target_ids)) + ')'    # [%s] * 지울 StudentID 개수 이후 ','를 기준으로 join
        cur.execute(sql, target_ids)

    conn.commit()
    return 

#### ※ 테이블 or 칼럼 이름에 ` (Backtick) 사용하는 이유: 예약어(ex. Order, etc.)가 아닌 테이블 or 칼럼으로 쓰기 위해

### 실습 (PyMySQL)

- db에 student 테이블을 생성해주세요.
    - id (빈 값 허용 안함, 자동 증가)
    - name (최대 16글자)
    - email (최대 32글자)
    - phone (최대 16글자)
    - major (최대 16글자)

In [114]:
with conn.cursor() as cur:
    sql = """CREATE TABLE `Student` (
    ID INT NOT NULL AUTO_INCREMENT,
    Name VARCHAR(16),
    Email VARCHAR(32),
    Phone VARCHAR(16),
    Major VARCHAR(16),
    PRIMARY KEY(ID)
    )"""
    cur.execute(sql)

conn.commit()

- 수강생 중 한 분의 데이터를 DB에 입력해주세요.

In [116]:
# 내 풀이
with conn.cursor() as cur:
    sql = 'INSERT INTO `Student` (`Name`, `Email`, `Phone`, `Major`) VALUES ("손준현", "leonica0429@gmail.com", "010123455678", "ISE")'
    cur.execute(sql)

conn.commit()

In [132]:
# 강사님 풀이
table_name = "Student"
columns = ['Name', 'Email', 'Phone', 'Major']
values = ("손준현", "leonica0429@gmail.com", "01012345678", "ISE")

with conn.cursor() as cur:
    sql = 'INSERT INTO {}({}) VALUES ({})'.format(table_name, ','.join(columns), ','.join(['%s'] * len(values)))
    cur.execute(sql, values)

conn.commit()

In [ ]:
# 함수화
def insert(table_name, columns, values):
    with conn.cursor() as cur:
        sql = 'INSERT INTO {}({}) VALUES ({})'.format(table_name, ','.join(columns), ','.join(['%s'] * len(values)))
        cur.execute(sql, values)

    conn.commit()

- 수강생 중 네 분의 데이터를 DB에 동시에 추가해주세요.

In [87]:
with conn.cursor() as cur:
    sql = 'INSERT INTO `Student` (`Name`) VALUES (%s)'
    cur.executemany(sql, [
        ("김도훈"),
        ("이정현"),
        ("주환희"),
        ("박정은")
    ])

conn.commit()

In [133]:
# 강사님 풀이
table_name = "Student"
columns = ['Name', 'Email', 'Phone', 'Major']
values = [
    ("김도훈", "abc123@gmail.com", "01011112222", "ABC"),
    ("이정현", "def123@gmail.com", "01033334444", "DEF"),
    ("주환희", "ghi123@gmail.com", "01055556666", "GHI"),
    ("박정은", "jkl123@gmail.com", "01077778888", "JKL")
]

with conn.cursor() as cur:
    sql = 'INSERT INTO {}({}) VALUES ({})'.format(table_name, ','.join(columns), ','.join(['%s'] * len(values[0])))
    cur.executemany(sql, values)

conn.commit()

In [ ]:
# 함수화
def insert_many(table_name:str, columns:list, values=list):
    with conn.cursor() as cur:
        sql = 'INSERT INTO {}({}) VALUES ({})'.format(table_name, ','.join(columns), ','.join(['%s'] * len(values[0])))
        cur.executemany(sql, values)

    conn.commit()

- 입력한 전체 데이터를 확인해주세요.

In [88]:
with conn.cursor() as cur:
    sql = 'SELECT * FROM Student'
    cur.execute(sql)
    data = cur.fetchall()

data

((1, '손준현', 'leonica0429@gmail.com', '010123455678', 'ISE'),
 (2, '김도훈', None, None, None),
 (3, '이정현', None, None, None),
 (4, '주환희', None, None, None),
 (5, '박정은', None, None, None))

In [134]:
# 강사님 풀이
columns = ['ID', 'Name']
columns = ['*']
table_name = "Student"

with conn.cursor() as cur:
    sql = "SELECT {} FROM {}".format(','.join(columns), table_name)
    cur.execute(sql)
    data = cur.fetchall()

data

((1, '손준현', 'leonica0429@gmail.com', '01012345678', 'ISE'),
 (2, '김도훈', 'abc123@gmail.com', '01011112222', 'ABC'),
 (3, '이정현', 'def123@gmail.com', '01033334444', 'DEF'),
 (4, '주환희', 'ghi123@gmail.com', '01055556666', 'GHI'),
 (5, '박정은', 'jkl123@gmail.com', '01077778888', 'JKL'))

- 한 수강생분의 이메일이 잘못 입력되었다고 가정하고, 이메일을 수정, DB에 반영해주세요.

In [94]:
with conn.cursor() as cur:
    sql = 'UPDATE `Student` SET `Email`=%s WHERE `ID`=%s'
    cur.execute(sql, ("abc123@gmail.com", 2))

conn.commit()

In [135]:
# 강사님 풀이
table_name = 'Student'
set_column = "Email"
set_value = "abc1234@gmail.com"
where_column = "ID"
where_value = 2

with conn.cursor() as cur:
    sql = 'UPDATE {} SET {}=%s WHERE {}={}'.format(table_name, set_column, set_value, where_column, where_value)
    cur.execute(sql, (set_value, ))

conn.commit()

- 수강생 한 분이 취업하셨습니다. DB에서 삭제 해주세요.

In [137]:
table_name = "Student"
where_column = "ID"
where_value = 4
with conn.cursor() as cur:
    sql = 'DELETE FROM {} WHERE {}={}'.format(table_name, where_column, where_value)
    cur.execute(sql)

conn.commit()

### 클래스화

- 생성 시 정보를 입력받아 DB에 연결하는 클래스를 생성해주세요.

In [123]:
class MariaDB:
    """
    MariaDB를 관리하는 클래스입니다.
    """

    def __init__(self, host, user, password, database, charset, port):
        self.host = host
        self.port = int(port)

        "다른 함수에서 사용하기 위해 self에 담아준다."
        try:
            self.conn = pymysql.connect(
                                        host=host,
                                        user=user,
                                        password=password,
                                        database=database,
                                        charset=charset,
                                        port=port)
            print("연결 성공")
        except Exception as e:
            print("연결 실패", e)
        
        return True
    
    def insert(self, table_name:str, columns:list, values:list):
        with self.conn.cursor() as cur:
            sql = 'INSERT INTO {}({}) VALUES ({})'.format(
                table_name, ','.join(columns), ','.join(['%s'] * len(values)))
            cur.execute(sql, values)

        self.conn.commit()
        return True
    
    def insert_many(self, table_name:str, columns:list, values:list):
        with self.conn.cursor() as cur:
            sql = 'INSERT INTO {}({}) VALUES ({})'.format(
                table_name, ','.join(columns), ','.join(['%s'] * len(values[0])))
            cur.executemany(sql, values)

        self.conn.commit()
        return True
    
    def select(self, table_name:str, columns:list, page=None, page_size=10):    # pagination 함수 추가
        if page:    # 페이지 있을 때,
            limit = page_size
            offset = page_size * (page - 1)
            with self.conn.cursor() as cur:
                sql = 'SELECT {} FROM {} LIMIT %s OFFSET %s'.format(
                    ','.join(columns), table_name)
                cur.execute(sql, (limit, offset))
                data = cur.fetchall()
        else:
            with self.conn.cursor() as cur:
                sql = 'SELECT {} FROM {}'.format(
                    ','.join(columns), table_name)         # SELECT "" FROM "" 부분 겹치므로 CONCAT()으로도 처리 가능
                cur.execute(sql)
                data = cur.fetchall()
        return data
    
    def update(self, table_name:str, set_column:list, set_value:str, where_column:str, where_value:str):
        with self.conn.cursor() as cur:
            sql = 'UPDATE {} SET {}=%s WHERE {}=%s'.format(
                table_name, set_column, where_column, where_value)
            cur.execute(sql, (set_value, where_value))

        self.conn.commit()
        return True

In [129]:
# 연결 테스트
db = MariaDB(**db_config)   # test
db1 = MariaDB(**db_config)  # production

연결 성공
연결 성공


In [ ]:
# insert 테스트
db.insert()

In [ ]:
# insert_many 테스트
db.insert_many()

In [ ]:
# select 테스트
db.select()

In [ ]:
# update 테스트
db.update()